# M2177.003100 Deep Learning <br> Assignment #4 Implementing Conditional Generative Adversarial Nets - part3 Labeld Face Data

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jaeyoon Yoo, November 2017

In this notebook, you will learn how to implement conditional Genverative Adversarial Nets (cGANs) <br>
The goal here is to build GANs that draw a face given its label. You can draw a black male/black female/white male/white female as you gives an input at the end of training. <br> 

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all parts**, run the *CollectSubmission.sh* script with your **Team number** as input argument. <br>
This will produce a zipped file called *[Your team number].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* &nbsp; Team_#)

### Some helpful tutorials and references for assignment #3:
- [1] TensorFlow official tutorials. [[link]](https://www.tensorflow.org/get_started/get_started)
- [2] Stanford CS231n lectures. [[link]](http://cs231n.stanford.edu/)
- [3] Goodfellow, Ian, et al. "Generative adversarial nets." Advances in neural information processing systems. 2014.
- [4] Mirza, Mehdi, and Simon Osindero. "Conditional generative adversarial nets." arXiv preprint arXiv:1411.1784 (2014).
- [5] Radford, Alec, Luke Metz, and Soumith Chintala. "Unsupervised representation learning with deep convolutional generative adversarial networks." arXiv preprint arXiv:1511.06434 (2015).

## 0. Download and load Face datasets
Unzip the face_dataset.tar.gz in data directory as follows.<br>
**cd ./data**<br>
**tar -xf face_dataset.tar.gz**<br>

Following is how to load the data. Modify  *data_dir* to be the directory the data is in. Or you will get an error.<br>

In [1]:
from utils import load_face
data_dir = './data/face_dataset'
im, label = load_face(data_dir)

There are 13143 images and corresponding lables. Labels have three values. Following is the detail:

Label1 - Male/Female : positive value means male<br>
Label2 - White/Not white : positive value means White<br>
Lable3 - Black/Not black : positive value menas black<br>
The large value represents the more property it has.

Note that the labels are not normalized and check the data by runing and modifying following code

In [ ]:
import pylab as plt
for i in range(5):
    plt.imshow(im[i])
    plt.show()
    print(label[i])
    print(im[i].shape)

## <a name="1"></a> 1. Build a network

In this section, you will implement neural networks for (1) generative model (2) discriminative model. You can reuse your code in part1 and improve it. Just write the code in whatever way you find most clear.

In [10]:
from utils import load_face
from utils import getNext_batch
from utils import save_images
from utils import vis_square
from utils import sample_label
from utils import sample_label_face
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2

from ops import conv2d
from ops import lrelu
from ops import de_conv
from ops import fully_connect
from ops import conv_cond_concat
from ops import batch_normal

import tensorflow as tf
import numpy as np

learning_rate = 0.0002
batch_size = 128
EPOCH = 100
loss_step    = 50
display_step = 50
sample_size = 100
y_dim = 3
channel = 3
output_size = 64

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

It is time for a generative model. You can change anything including the argument If you need. Feel free to change it and improve it.

In [5]:
def gern_net(batch_size, z , y ,sample_size, y_dim, output_size):
    #### TODO ####
    with tf.variable_scope('gen'):
        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
        b_init = tf.constant_initializer(0.0)

        # concat layer
        z_input = tf.reshape(z, [-1, 1, 1, sample_size])
        y_label = tf.reshape(y, [-1, 1, 1, y_dim])
        inputs = tf.concat([z, y], 1)
        
        # project layer
        proj_size = 4*4*1024
        W = tf.get_variable("W0", [sample_size + y_dim, proj_size], initializer=w_init)
        b = tf.get_variable("b0", [proj_size], initializer=b_init)
        projected = lrelu(batch_normal(fully_connect(inputs, W, b), scope='bn0'))
        reshape = tf.reshape(projected, [batch_size, 4, 4, 1024])

        # 1st hidden layer
        W = tf.get_variable("W1", [5, 5, 512, 1024], initializer=w_init)
        b = tf.get_variable("b1", [512], initializer=b_init)
        deconv1 = lrelu(batch_normal(de_conv(reshape, W, b, [batch_size, 8, 8, 512]), scope='bn1'))

        # 2nd hidden layer
        W = tf.get_variable("W2", [5, 5, 256, 512], initializer=w_init)
        b = tf.get_variable("b2", [256], initializer=b_init)
        deconv2 = lrelu(batch_normal(de_conv(deconv1, W, b, [batch_size, 16, 16, 256]), scope='bn2'))
        
        # 3nd hidden layer
        W = tf.get_variable("W3", [5, 5, 128, 256], initializer=w_init)
        b = tf.get_variable("b3", [128], initializer=b_init)
        deconv3 = lrelu(batch_normal(de_conv(deconv2, W, b, [batch_size, 32, 32, 128]), scope='bn3'))

        # output layer
        W = tf.get_variable("W4", [5, 5, 3, 128], initializer=w_init)
        b = tf.get_variable("b4", [3], initializer=b_init)
        deconv3 = de_conv(deconv3, W, b, [batch_size, 64, 64, 3])
        o = tf.nn.tanh(deconv3)

        return o
    

Now, it's time for a discriminative model. Again, you can change anything if you need

In [6]:
def dis_net(data_array , y, batch_size, y_dim, reuse=False):
    #### TODO ####
    with tf.variable_scope('dis', reuse=reuse):
        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
        b_init = tf.constant_initializer(0.0)

        yb = tf.reshape(y, [batch_size, 1, 1, y_dim])
        x = conv_cond_concat(data_array, yb)

        # 1st hidden layer
        W = tf.get_variable("W1", [5, 5, x.get_shape()[-1], channel+y_dim], initializer=w_init)
        b = tf.get_variable("b1", [channel+y_dim], initializer=b_init)
        conv1 = lrelu(conv2d(x, W, b))
        
        # 2nd hidden layer
        W = tf.get_variable("W2", [5, 5, conv1.get_shape()[-1], 64], initializer=w_init)
        b = tf.get_variable("b2", [64], initializer=b_init)
        conv2 = lrelu(batch_normal(conv2d(conv1, W, b), scope='bn2'))
        conv2_flatten = tf.reshape(conv2, [batch_size, -1])
        
        # 3rd hidden layer
        W = tf.get_variable("W3", [conv2_flatten.get_shape()[-1], 1024], initializer=w_init)
        b = tf.get_variable("b3", [1024], initializer=b_init)
        out3 = lrelu(batch_normal(fully_connect(conv2_flatten, W, b), scope='bn3'))

        # output layer
        W = tf.get_variable("W4", [1024, 1], initializer=w_init)
        b = tf.get_variable("b4", [1], initializer=b_init)
        logits = fully_connect(out3, W, b)
        o = tf.nn.sigmoid(logits)

        return o, logits


## <a name="2"></a> 2. Build a main part and train it

In this section, you will implement the main part. Then run the code and check the model draws the face properly.

When you are done, run the following to check your implementations.

Following code will make 'samples_for_test' directory that resulting image will be saved in. You can change the directory as you want.

In [7]:
from utils import load_mnist
from utils import save_images
from utils import vis_square
from utils import sample_label
from utils import getNext_batch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os

sample_dir = 'samples_for_test_faces'

if os.path.exists(sample_dir) == False:
    os.makedirs(sample_dir)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Feel free to fill in the main part. You can copy the part1 code or write your own code.

Your goal is to **generate 4 row and 8 column images(32 total)**.

**Each row should correspond to each label**.

First row : black male<br>
Second row: black female<br>
Third row : white male<br>
Fourth row: white female<br>

You can use "save_images" method in *utils.py* to align generated image by 4*8. See part1 code to get how to use it.

You must show **at least three generated images**. (At the beginning of ,in the midway of, at the end of training.)


In [ ]:
#### TODO ####
def new_sample_label_face(num):
    label_vector = np.zeros((num,3))
    for i in range(32):
        if i < 8:
            label_vector[i,0]=2  # male
            label_vector[i,1]=-2 # not white
            label_vector[i,2]=2  # black
        elif i < 16:
            label_vector[i,0]=-2 # female
            label_vector[i,1]=-2 # not white
            label_vector[i,2]=2  # black
        elif i < 24:
            label_vector[i,0]=2  # male
            label_vector[i,1]=2  # white
            label_vector[i,2]=-2  # not black
        else:
            label_vector[i,0]=-2  # female
            label_vector[i,1]=2 # white
            label_vector[i,2]=-2  # not black
    return label_vector

tf.reset_default_graph()

sample_z = np.random.uniform(-1 , 1 , size = [batch_size , sample_size])

y = tf.placeholder(tf.float32, [None , y_dim])

images = tf.placeholder(tf.float32, [batch_size, output_size, output_size, channel])

z = tf.placeholder(tf.float32, [None , sample_size])

fake_images = gern_net(batch_size, z , y ,sample_size, y_dim,output_size)

##the loss of gerenate network
D_pro , D_logits = dis_net(images, y , batch_size, y_dim,  False)

G_pro, G_logits = dis_net(fake_images , y , batch_size, y_dim, True)

# DEFINE LOSS FUNCTION #

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits, labels=tf.ones_like(D_logits)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=G_logits, labels=tf.zeros_like(G_logits)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=G_logits, labels=tf.ones_like(G_logits)))

#############

t_vars = tf.trainable_variables()

d_var = [var for var in t_vars if 'dis' in var.name]
g_var = [var for var in t_vars if 'gen' in var.name]

opti_D = tf.train.AdamOptimizer(learning_rate=learning_rate , beta1=0.5).minimize(D_loss , var_list=d_var)
opti_G = tf.train.AdamOptimizer(learning_rate=learning_rate , beta1=0.5).minimize(G_loss , var_list=g_var)

init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:

    sess.run(init)
    e = 0
    step = 0

    while e <= EPOCH:
        batch_num = 0
        while batch_num < len(im) / batch_size - 1:

            step = step + 1

            realbatch_array , real_labels = getNext_batch(im , label , batch_num, batch_size)
            
            #Get the z
            batch_z = np.random.uniform(-1 , 1 , size=[batch_size , sample_size])

            _ = sess.run(opti_D, feed_dict={images:realbatch_array, z:batch_z , y:real_labels})
            _ = sess.run(opti_G, feed_dict={z: batch_z , y:real_labels})

            batch_num += 1

            if step % loss_step == 0:

                d_get_loss = sess.run(D_loss , feed_dict = {images:realbatch_array , z:batch_z , y:real_labels})
                g_get_loss = sess.run(G_loss , feed_dict = {z: batch_z , y:real_labels})
                print("EPOCH %d step %d: D: loss = %.7f G: loss=%.7f " % (e , step , d_get_loss , g_get_loss))

            #if np.mod(step , display_step) == 1:
        if np.mod(e , 10) == 0:

            sample_images = sess.run(fake_images , feed_dict={z:sample_z , y:new_sample_label_face(batch_size)})
            sample_images = sample_images[:32,:,:,:]
            image_name = './{}/train_{:02d}_{:04d}.png'.format(sample_dir , e , step)
            save_images(sample_images , [4,8] , image_name)
        
            img = mpimg.imread(image_name)
            imgplot = plt.imshow(img)
            plt.show()

        e = e + 1
        batch_num = 0

EPOCH 0 step 50: D: loss = 1.3996160 G: loss=0.6708941 
